## Домашнее задание №3

### Укажите ФИО и группу

Добрый день,  

Вам предлагается ответить на ряд вопросов для того, чтобы закрепить пройденный материал да и просто хорошо и с некоторой пользой провести несколько часов...  
  
Код, который вы пришлете, должен быть рабочим...

### Импортируем необходимые модули 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/Classroom/Методы машинного обучения/DA_ML_20')

In [ ]:
import numpy as np
import scipy, warnings, copy
import pandas as pd
import matplotlib.pyplot as plt # библиотека для построения графиков
import matplotlib.ticker as mticker
import seaborn as sns # библиотека для построения графиков

from sklearn import preprocessing, decomposition, manifold, model_selection, pipeline, cluster, datasets, mixture, metrics
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, roc_curve, precision_score, roc_auc_score, f1_score

from sklearn import linear_model as lm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

from sklearn.datasets import make_regression, make_classification
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV, KFold

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC, SVC

from sklearn.preprocessing import StandardScaler

%matplotlib inline 

In [ ]:
## функция для генерации "предсказаний" с заданным уровнем ошибок 
def make_errors(Y, frac = .3, seed = 42):
    """
    Y - our "true" binary labels
    frac - the "desired" error rate
    """
    np.random.seed(seed)
    ## выберем индексы ответов, которые у нас будут ошибочными
    ix = np.random.choice(range(len(Y)), replace=False, size =int(len(Y) * frac)) 
    Ys = copy.copy(Y) ## создадим копию массива с ответами, чтобы не изменять оригинальные ответы
    Ys[ix] = np.abs(Y[ix] - 1) ## инвертируем метки классов
    return Ys

### В этом задании у вас три сгенерированных заранее датасета, остальные есть в этом ноутбуке. В этом домашнем задании 5 пунктов + 1 бонусный. Всего за эту домашку можно получить 10 баллов.

Рекомендую внимательно ознакомиться с материалами, которые я вам выдал, посмотреть ссылки на дополнительные материалы, а для заинтересованных в теме &mdash; пройти дополнительные курсы по анализу данных и машинному обучению. Прошу обратить внимание на задания и вопросы, которые изредка, но встречаются в ноутбуках данного курса. Лучше их все-таки выполнить. Если не можете что-то сделать самостоятельно &mdash; спрашивайте.

### 1. Оценка классификаторов (2 балла).

1. В первой части задания вам необходимо работать с заранее сгенерированными массивами предсказанных и настоящих меток классов (Y_predicted, Y_true). Необходимо:

    * Построить для них матрицу ошибок (confusion matrix)
    * Определить Accuracy, Precision, Recall, F1-метрику
    * Построить график ROC, определите значение AUC
    * Проверить, можно ли улучшить качество предсказания метки класса? Верно ли подобрано пороговое значение? (бонусный пункт)
    
    
2. Во второй части задания вам будет дан набор, в котором есть 200 объектов: 20 из класса 1 и 180 из класса 0. Для него будут доступны новые истинные метки классов, а также результаты предсказания нескольких алгоритмов. От вас требуется:

    * Определить accuracy для каждого из методов. Какой из них лучше? Определить это также и для F1-метрики.
    * Посчитать метрики accuracy и F1 для консенсусных предсказаний (пояснения в коде). Изменилось ли что-нибудь?
    * Какой вывод вы можете сделать из данного упражнения?

#### Первая часть

In [ ]:
## результаты "предсказания"
# np.random.seed(0) ## зафиксируем random seed
# Yp = np.random.uniform(0, 0.9, 100) ## синтезируем ответы
Y_predicted = np.array([0.49393215, 0.64367043, 0.54248704, 0.49039486, 0.38128932, 0.5813047 , 0.39382849, 0.8025957 ,
            0.86729648, 0.34509737, 0.71255253, 0.47600543, 0.5112401 , 0.83303697, 0.06393245, 0.07841637,
            0.01819656, 0.74935786, 0.70034108, 0.78301093, 0.88075651, 0.71924271, 0.41533143, 0.70247626, 
            0.10644698, 0.57592892, 0.12901796, 0.85020203, 0.46966349, 0.37319575, 0.23810005, 0.69681032, 
            0.4105353 , 0.51159055, 0.01691082, 0.55587195, 0.55088615, 0.5552406 , 0.84937327, 0.61363827,
            0.32355711, 0.39332876, 0.62786808, 0.05420292, 0.60009004, 0.60357408, 0.1893443 , 0.11603367,
            0.28388552, 0.32733969, 0.51317709, 0.39474136, 0.88953645, 0.09184033, 0.18798908, 0.14517857,
            0.58779749, 0.22796244, 0.4196797 , 0.21998303, 0.14307263, 0.09933763, 0.59069663, 0.12436466,
            0.17692413, 0.33185265, 0.73889391, 0.08739115, 0.75415042, 0.08648857, 0.87881352, 0.42178608,
            0.87908498, 0.54436097, 0.66533722, 0.03526901, 0.25452627, 0.10817691, 0.26652618, 0.10685495,
            0.28618486, 0.3728367 , 0.05773275, 0.62322491, 0.50994131, 0.23885054, 0.47092325, 0.08454646,
            0.51835185, 0.83636658, 0.28671206, 0.60066934, 0.11861808, 0.64469448, 0.26046548, 0.16487223,
            0.52786164, 0.01809679, 0.74604603, 0.00422593])

## настоящие ответы - метки классов
Y_true = np.array([1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1,
            0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1,
            1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
            1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
            0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0], dtype=np.int64)

In [ ]:
## 1. Постройте матрицу ошибок -- confusion matrix
## Тут должен быть ваш код

In [ ]:
## 2. Определите Accuracy, Precision, Recall, F1-метрику
#accuracy_score
#precision_score
#recall_score
#f1_score
## Тут должен быть ваш код

In [ ]:
## 3. Постройте график ROC, определите значение AUC
## для примера смотрите ноутбук №3
## Тут должен быть ваш код

In [ ]:
## * 4. Проверьте, можно ли улучшить качество предсказания метки класса? Верно ли подобрано пороговое значение?
## Тут должен быть ваш код

#### Вторая часть 

In [ ]:
## Вам дан набор, в котором есть 200 объектов: 20 из класса 1 и 180 из класса 0
## Y   - истинные метки классов
## Yp1 - результаты предсказания алгоритма 1
## Yp2 - результаты предсказания алгоритма 2
## Yp3 - результаты предсказания алгоритма 3
## Yp4 - результаты предсказания алгоритма 4
np.random.seed(42)
Y =  np.hstack( [ np.ones(20), np.zeros(180) ] ) 
np.random.shuffle(Y)
Yp1 = make_errors(Y, 0.25, seed = 9)
Yp2 = make_errors(Y, 0.25, seed = 42)
Yp3 = make_errors(Y, 0.25, seed = 1024)
Yp4 = np.zeros(200)

In [ ]:
## Определите accuracy каждого из методов
## <ваш код>

In [ ]:
## Какой из методов лучше?
## <ваш комментарий>

In [ ]:
## Повторите все для метрики F1

## <ваш код>
## <ваш комментарий>

In [ ]:
## Используйте результаты моделей №1-3 -- возьмем и усредним результаты их предсказаний:
Yc = ((Yp1 + Yp2 + Yp3)/3 > 0.5).astype(int)

In [ ]:
## Оцените результат по метрикам Accuracy и F1, стал ли он лучше? Почему?

In [ ]:
## Можете добавить к вычислению консенсуса результат Yp4. Определите метрики Accuracy и F1. Что изменилось? 

In [ ]:
## Какой вывод вы можете сделать из данного упражнения?

### 2. Исследование корреляций (1 балл).

В вашем распоряжении оказался некий датасет (наблюдения за экспрессией некоторых генов, рекомендательная база данных для ассортимента интернет-маркета, относительный расход электричества в домах в некотором городе и т.д.). Вам необходимо:

1. Посмотреть корреляцию датасета с целевой переменной. Есть ли такие признаки, абсолютное значение коэффициента корреляции для которых > 0.5?
2. Посмотреть корреляцию между признаками внутри датасета. Какое распределение у абсолютных значений коэффициентов корреляции Пирсона для первого признака? Много ли там признаков, абсолютное значение коэффициента корреляции для которых > 0.5?
3. Увеличить/уменьшить количество наблюдений N = 10, N = 100
4. Уменьшить количество признаков в данных: D = 100, 500, 5000
5. Дать ответы на вопросы:
    - Чем обусловлены наши проблемы с этим набором данных? 
    - Как можно бороться со случайными корреляциями?
    - Почему при тестировании нескольких статистических гипотез необходимо проводить поправку на множественную проверку гипотез?

In [ ]:
np.random.seed(42)
N = 30    ## количество наблюдений
D = 10000 ## размерность наших данных -- например, число некоторых генов, экспрессию которых мы изучаем
Y = pd.Series(np.random.normal(0, 1, N)) ## это какие-то наши целевые значения
X = pd.DataFrame( np.random.normal(0, 1, (N, D)) ) ## это наши независимые переменные, скажем экспрессия некоторых генов
## как мы видим все эти данные абсолютно случайные...

In [ ]:
## можем ли мы получить высокую корреляцию с нашей целевой переменной?
ccY = X.corrwith(Y)
## посмотрите на распределение значений. Есть ли такие "гены", абсолютное значение коэффициента корреляции для которых > 0.5
## <ваш код>

In [ ]:
## теперь давайте проведем аналогичный эксперимент для нашей матрицы X... 
## определим "коэкспрессию" ...
ccX = X.corr()

In [ ]:
## Посмотрите на респределение абсолютных значений коэффициентов корреляции Пирсона
## Например для первого гена из нашего массива данных:
ccX.iloc[0,1:]
## много ли получилось генов с коэфф. корреляции > 0.5?

In [ ]:
## Попробуйте увеличить/уменьшить количество наблюдений N = 10, N = 100

In [ ]:
## Попробуйте использовать меньшее количество переменных: D = 100, 500, 5000

In [ ]:
## Чем обусловлены наши проблемы с этим набором данных? Как можно бороться со случайными корреляциями?

In [ ]:
## Почему при тестировании нескольких статистических гипотез необходимо проводить поправку на множественную проверку гипотез?

Тем, для кого выражение "p-hacking" незнакомо -- обязательно смотреть статьи: 
* https://en.wikipedia.org/wiki/Data_dredging
* https://ru.wikipedia.org/wiki/Поправка_на_множественную_проверку_гипотез
* http://www.machinelearning.ru/wiki/index.php?title=Множественная_проверка_гипотез

Для развлечения:  
http://www.tylervigen.com/spurious-correlations

### 3. Полиномиальная регрессия (1 балл).

Вам предлагается самим поэксперементировать с построением полиномиальной регрессией, способной приблизить функцию **myy**. Постройте модели для нескольких степеней (1, 2, 4, 15) и определите среднеквадратичную ошибку предсказания. Какая модель является на ваш взгляд оптимальной? Почему? С какими проблемами мы столкнулись при построении моделей (что иллюстрирует этот эксперимент)?

In [ ]:
from sklearn import linear_model as lm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
def myy(x):
    return np.cos(1.5 * np.pi * x)

def poly_lm(X,y, dg):
    polynomial_features = PolynomialFeatures(degree=dg,
                                             include_bias=False)
    linear_regression = lm.LinearRegression()
    pipeline = Pipeline([("polynomial_features", polynomial_features),
                         ("linear_regression", linear_regression)])
    pipeline.fit(X[:, np.newaxis], y)
    return pipeline

np.random.seed(0)
X = np.random.rand(70) ## набор тренировочных данных X
y = myy(X) + np.random.randn(70) * 0.1 ## набор истинных значений y для тренировочного набора

X_test = np.linspace(-.01, 1.01, 30)  ## набор тестовых данных X
y_test = myy(X_test) ## набор истинных значений y для тестового набора

In [ ]:
## получим модель для степени = 1, 2, 4, 15
mod1 = poly_lm( X, y, dg = 1  )
## выполним предсказание для тестового набора
y_t1 = mod1.predict(X_test[:, np.newaxis])
## определите среднеквадратичную ошибку предсказания для тестового набора и для тренировочного набора
# metrics.mean_squared_error(y_true, y_predicted)
## <ваш код>

In [ ]:
## аналогично поступите для степени 2, 4 и 15
## <ваш код>

In [ ]:
## Какая модель является на ваш взгляд оптимальной? Почему? С какими проблемами мы столкнулись при построении моделей?

### 4. Шкалирование данных (2 балла).

Для этого задания часть датасетов генерируется в коде, часть - приложена в виде файла к вашему ДЗ. Необходимо ответить на следующие вопросы:
1. Зачем нужно проводить шкалирование данных? 
2. Всегда ли это нужно делать?
3. Какой/какие из предоставленных вам наборов надо шкалировать? Обоснуйте свой ответ при помощи кода.

In [ ]:
X1, Y1 = make_classification(n_samples=500, n_features=50, 
                             n_informative=20, n_redundant=10, 
                             n_repeated=0, n_classes=3, 
                             n_clusters_per_class=1, 
                             weights=None, # The proportions of samples assigned to each class. 
                                           # If None, then classes are balanced.
                             flip_y=0.01, class_sep=1.0, 
                             hypercube=True,
                             shift=1.0, 
                             scale=None, 
                             shuffle = False,
                             random_state=42)

X2, Y2 = make_classification(n_samples=500, n_features=50, 
                             n_informative=20, n_redundant=10, 
                             n_repeated=0, n_classes=3, 
                             n_clusters_per_class=1, 
                             weights=None, # The proportions of samples assigned to each class. 
                                           # If None, then classes are balanced.
                             flip_y=0.01, class_sep=1.0, 
                             hypercube=True,
                             shift=1.0, 
                             scale=1, 
                             shuffle = False,
                             random_state=42)

XY3 = pd.read_csv('X3.csv')
X3, Y3 = XY3.iloc[:,1:], XY3.Y

In [ ]:
## <ваш код>

### 5. Неразмеченные данные (2 балла).

Вам дан набор неразмеченных данных X4. Проведите его анализ с помощью PCA, визуализируйте результат. Какие выводы вы можете сделать? Чем может быть обусловлена такая картина? Как стоит поступить? Можете сразу проверить свою гипотезу.

In [ ]:
X4 = pd.read_csv('X4.csv')

In [ ]:
## Проведите его анализ с помощью PCA. 
## Визуализируйте результат.
## <Ваш код>

In [ ]:
## Какие выводы вы можете сделать? Чем может быть обусловлена такая картина? Как стоит поступить?
## Ваши комментарии

### 6. Серийные эффекты (2 балла).

Здесь мы наглядно познакомимся с тем, что такое серийные эффекты (batch effect). 
Вам дан набор данных 'expr_data.tsv'. 
1. Какая переменная в вашем наборе данных указывает на возможное присутствие серийных эффектов? С чем могут быть связаны серийные эффекты?
2. Ваша задача попытаться найти маркерные признаки. Имеет ли ваша задача решение?
3. Какие проблемы вы заметили в данных? Можно ли с ними бороться? Имеет ли это смысл в данном случае?

Обязательно посмотрите на все переменные. Можете провести PCA, покрасить точки по классам, по каким-то интересным факторам...

In [ ]:
## <здесь должен быть ваш код>